In [1]:
#Please  construct  a  new  dataset  by  either  adding  two  independent  variables  or  removing  two independent
#variables  from  finalsample.dta  dataset.  If  you  choose  to  add  two  independent variables, you could add
#any two independent variables that you think help explain stock returns. If  you  choose  to  remove  two  
#independent  variables,  you  could  remove  any  two  independent variables that already exist in the 
#finalsample.dta dataset.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
plt.rcParams['figure.figsize'] = [20, 15]

In [2]:
nndata=pd.read_stata('/Users/jimmyaspras/Downloads/finalsample.dta')
nndata.columns

Index(['gvkey', 'datadate', 'sic_2', 'lagdate', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'lagdatadate', 'atq', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'dvpspq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2'],
      dtype='object')

In [3]:
nndata.sort_values(by=['datadate'], inplace=True)
nndata1=nndata[nndata['lagPrice2']>=5]#remove penny stocks
nndata1['Year']=nndata1['datadate'].dt.year
nndata1['Month']=nndata1['datadate'].dt.month
#set gvkey and datadate as the index
nndata1=nndata1.set_index(['gvkey','datadate'])
nndata1.head()

<ipython-input-3-e895412330a8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nndata1['Year']=nndata1['datadate'].dt.year
<ipython-input-3-e895412330a8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nndata1['Month']=nndata1['datadate'].dt.month


,,sic_2,lagdate,lagRet2,lagVOL2,lagPrice2,lagMV2,lagShareturnover2,lagRet2_sic,lagRet12,lagVOL12,...,BM,PE,div_p,loglagPrice2,loglagVOL12,loglagMV2,logatq,loglagVOL2,Year,Month
gvkey,datadate,,,,,,,,,,,,,,,,,,,,,
1004.0,2001-04-30,50.0,2001-02-28,-0.104674,1129800.0,13.600,366.27520,0.041950,-0.048423,0.052613,1481250.0,...,0.931715,85.000000,0.006250,2.610070,14.208397,5.903385,6.650203,13.937551,2001,4
30915.0,2001-04-30,33.0,2001-02-28,-0.050095,63300.0,10.050,91.80675,0.006929,0.004113,-0.018221,239500.0,...,1.624488,37.222222,0.000000,2.307573,12.386309,4.519686,5.489338,11.055641,2001,4
31170.0,2001-04-30,33.0,2001-02-28,-0.079365,296200.0,7.250,44.02925,0.048773,0.004113,-0.005784,345900.0,...,1.076057,34.523810,0.000000,1.981001,12.753905,3.784854,4.315018,12.598790,2001,4
31600.0,2001-04-30,33.0,2001-02-28,-0.035294,1058400.0,5.125,84.70600,0.064037,0.004113,0.078186,1158450.0,...,3.812655,36.607143,0.009756,1.634131,13.962593,4.439186,6.527946,13.872269,2001,4
61635.0,2001-04-30,33.0,2001-02-28,0.044444,201200.0,11.750,76.35150,0.030963,0.004113,0.318682,341450.0,...,1.379685,28.658537,0.000000,2.463853,12.740957,4.335348,5.667520,12.212055,2001,4


In [4]:
#Split  your  new  dataset  into  training  and  testing  samples.  Testing  sample  should  include  data with
#year>=2016. 
#
#Drop dvpspq and atq from the train/test data
#
train=nndata1[nndata1['Year']<2016]
X_train=train[['sic_2', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2']]

In [5]:
#Set return as the dependent training variable
Y_train=train[['ret']]

In [6]:
#Set testing independent variables
test=nndata1[nndata1['Year']>=2016]
X_test=test[['sic_2', 'lagRet2', 'lagVOL2',
       'lagPrice2', 'lagMV2', 'lagShareturnover2', 'lagRet2_sic', 'lagRet12',
       'lagVOL12', 'lagShareturnover12', 'lagRet12_std', 'lagRet12_min',
       'lagRet12_max', 'lagRet12_sic', 'ceqq', 'cheq',
       'dlttq', 'epspiq', 'saleq', 'sp500_ret_d', 'nasdaq_ret_d',
       'r2000_ret_d', 'dollar_ret_d', 'VIX', 'yield_3m', 'yield_10y',
       'gdp_growth', 'Bull_ave', 'Bull_Bear', 'ret', 'debt', 'cash', 'sale',
       'BM', 'PE', 'div_p', 'loglagPrice2', 'loglagVOL12', 'loglagMV2',
       'logatq', 'loglagVOL2']]

In [7]:
#Set return as the dependent testing variable
Y_test=test[['ret']]

In [8]:
#Calculate avg monthly risk free return
rf1=pd.read_excel("/Users/jimmyaspras/Downloads/Treasury bill.xlsx")
rf1['rf']=rf1['DGS3MO']/1200
rf2=rf1[['Date','rf']].dropna()
rf2['Year']=rf2['Date'].dt.year
rf2['Month']=rf2['Date'].dt.month
rf3=rf2[['Year','Month','rf']].groupby(['Year','Month'], as_index=False).mean()

In [9]:
#Import benchmark index return
indexret1=pd.read_stata("/Users/jimmyaspras/Downloads/Index return.dta")
#Import factors data
Factor=pd.read_excel("/Users/jimmyaspras/Downloads/Factors.xlsx")

In [10]:
#Build  a  neural  network  with  one  hidden  layer  and  20  neurons  in  the  hidden  layer.  Set  batch 
#size=10,000.  Use  GridSearchCV  to  search  for  the  best  value  of  epochs  among  [10,  20,  30,  40]. Use  
#the  best value  of  epochs  found  in  the  search  to  train  this  neural  network  using  your  new training 
#sample. Use the trained neural network to predict returns based on your new testing sample. Report the average 
#return of the portfolio that consists of the 100 stocks with the highest predicted returns in each year-month. 
#Also, report the Sharpe ratio of the portfolio. 
def shallownetwork():
    model=Sequential()
    model.add(Dense(20,kernel_initializer='uniform',activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='Adam')
    return model

In [11]:
tsplit=TimeSeriesSplit(n_splits=5,test_size=50000,gap=5000)
model1=KerasRegressor(build_fn=shallownetwork)
param_candidate={'epochs': [10,20,30,40]}
grid=GridSearchCV(estimator=model1,param_grid=param_candidate,n_jobs=-1,cv=tsplit,scoring='neg_mean_squared_error')
grid.fit(X_train,Y_train,batch_size=10000,verbose=0)
grid.cv_results_

{'mean_fit_time': array([10.97182283,  8.14502058, 12.20729866, 16.87534151]),
 'std_fit_time': array([2.02224683, 1.5556716 , 1.79913007, 4.94902274]),
 'mean_score_time': array([3.55519547, 4.03498945, 4.05364237, 3.30667205]),
 'std_score_time': array([1.14943611, 0.70416456, 1.47834758, 2.14423576]),
 'param_epochs': masked_array(data=[10, 20, 30, 40],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'epochs': 10}, {'epochs': 20}, {'epochs': 30}, {'epochs': 40}],
 'split0_test_score': array([-3.15814442e+09, -1.45939508e+09, -8.41260678e+08, -1.47120575e+07]),
 'split1_test_score': array([-1.42508775e+10, -5.88534408e+08, -8.67318107e+07, -7.57039395e+09]),
 'split2_test_score': array([-6.71055384e+09, -2.37823606e+09, -2.70078202e+08, -2.63812739e+07]),
 'split3_test_score': array([-2.39900563e+08, -2.68398658e+08, -4.05095026e+06, -2.62379774e+07]),
 'split4_test_score': array([-1.70744964e+08, -1.21994167e+08, -8.9

In [12]:
grid.best_params_

{'epochs': 30}

In [13]:
nnretrain=shallownetwork()
#**grid.best_params_ feeds epochs directly into the model fitting
nnretrain.fit(X_train,Y_train,**grid.best_params_,batch_size=10000,verbose=1)

Epoch 1/30
56/56 [==============================] - 1s 8ms/step - loss: 2066401853440.0000
Epoch 2/30
56/56 [==============================] - 0s 9ms/step - loss: 23360057344.0000
Epoch 3/30
56/56 [==============================] - 0s 8ms/step - loss: 7124564992.0000
Epoch 4/30
56/56 [==============================] - 0s 7ms/step - loss: 5289424896.0000
Epoch 5/30
56/56 [==============================] - 0s 9ms/step - loss: 3722147584.0000
Epoch 6/30
56/56 [==============================] - 0s 5ms/step - loss: 2614666752.0000
Epoch 7/30
56/56 [==============================] - 0s 5ms/step - loss: 1806205312.0000
Epoch 8/30
56/56 [==============================] - 0s 5ms/step - loss: 1949585792.0000
Epoch 9/30
56/56 [==============================] - 0s 5ms/step - loss: 1245196928.0000
Epoch 10/30
56/56 [==============================] - 0s 5ms/step - loss: 849846848.0000
Epoch 11/30
56/56 [==============================] - 1s 10ms/step - loss: 687818432.0000
Epoch 12/30
56/56 [========

In [14]:
Y_predict=pd.DataFrame(nnretrain.predict(X_test),columns=['Y_predict'])

In [15]:
Y_test1=pd.DataFrame(Y_test).reset_index()

In [16]:
Comb1=pd.merge(Y_test1, Y_predict, left_index=True,right_index=True,how='inner')
Comb1['Year']=Comb1['datadate'].dt.year
Comb1['Month']=Comb1['datadate'].dt.month
rank1=Comb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
rank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
stock_long1=pd.merge(Comb1,rank1,left_index=True, right_index=True)
stock_long2=stock_long1[stock_long1['Y_predict_rank']<=100]
stock_long2['datadate'].value_counts()
stock_long3=stock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()
stock_long4=pd.merge(stock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5=pd.merge(stock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5['ret_rf']=stock_long5['ret']-stock_long5['rf']
stock_long5['ret_sp500']=stock_long5['ret']-stock_long5['sp500_ret_m']
stock_long5=sm.add_constant(stock_long5)
sm.OLS(stock_long5[['ret']],stock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 12 Aug 2021   Prob (F-statistic):                nan
Time:                        21:01:25   Log-Likelihood:                 85.629
No. Observations:                  64   AIC:                            -169.3
Df Residuals:                      63   BIC:                            -167.1
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0149      0.008      1.876      0.065      -0.001       0.031
==============================================================================
Omnibus:                       16.400   Durbin-Watson:                   2.169
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.856
Skew:                          -0.580   Prob(JB):                     4.06e-11
Kurtosis:                       7.074   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [ ]:
#Return of 1.49% over market

In [17]:
Ret_rf=stock_long5[['ret_rf']]
SR=(Ret_rf.mean()/Ret_rf.std())*np.sqrt(12)
SR

ret_rf    0.755798
dtype: float64

In [18]:
#Build a deep neural network with more than 2 hidden layers. Feel free to pick the number of hidden layers. Use 
#RandomizedSearchCV to search for the best values of epochs, batch size, and the  number  of  neurons  in  each  
#hidden  layer.  Use  the  best  values  of  epochs,  batch  size,  and  the number of neurons in each hidden layer
#found in the search to train the deep neural network using your new training sample. Use the trained deep neural 
#network to predict returns based on your new testing sample. Report the average return of the portfolio that 
#consists of the 100 stocks with the highest predicted returns in each year-month. Also, report the Sharpe ratio 
#of the portfolio.  

In [19]:
def deepnetwork(no_neuron1,no_neuron2,no_neuron3,no_neuron4,no_neuron5,no_neuron6,no_neuron7,no_neuron8,no_neuron9,no_neuron10):
    model=Sequential()
    model.add(Dense(no_neuron1, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron2, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron3, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron4, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron5, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron6, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron7, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron9, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(no_neuron10, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='Adam')
    return model

In [20]:
deepnetworktsplit=TimeSeriesSplit(n_splits=5, test_size=50000, gap=5000)

In [21]:
param_dist = {'no_neuron1': randint(10,100),
              'no_neuron2': randint(10,100),
              'no_neuron3': randint(10,100),
              'no_neuron4': randint(10,100),
              'no_neuron5': randint(10,100),
              'no_neuron6': randint(10,100),
              'no_neuron7': randint(10,100),
              'no_neuron8': randint(10,100),
              'no_neuron9': randint(10,100),
              'no_neuron10': randint(10,100),
              'epochs': randint(10,50),
              'batch_size': randint(10000,50000)}

In [22]:
model2 = KerasRegressor(build_fn=deepnetwork)
rgrid = RandomizedSearchCV(estimator=model2, param_distributions=param_dist, n_iter=5, cv=deepnetworktsplit,
                           scoring='neg_mean_squared_error', n_jobs=-1)
rgrid.fit(X_train,Y_train,verbose=0)
rgrid.cv_results_

{'mean_fit_time': array([166.63865571, 180.74396281,  90.85305777, 157.7881814 ,
         97.5726676 ]),
 'std_fit_time': array([24.30869409, 31.50696439, 17.61875564, 28.10562476,  5.55643238]),
 'mean_score_time': array([1.85407853, 2.13774819, 1.82200356, 1.93170042, 1.2208168 ]),
 'std_score_time': array([0.23885812, 0.18154211, 0.27361068, 0.4278632 , 0.59177206]),
 'param_batch_size': masked_array(data=[38088, 33355, 15925, 13336, 11593],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[32, 47, 13, 38, 33],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_no_neuron1': masked_array(data=[10, 21, 73, 40, 43],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_no_neuron10': masked_array(data=[31, 11, 26, 20, 92],
              mask=[False, False, False, False,

In [23]:
rgrid.best_params_

{'batch_size': 38088,
 'epochs': 32,
 'no_neuron1': 10,
 'no_neuron10': 31,
 'no_neuron2': 19,
 'no_neuron3': 62,
 'no_neuron4': 34,
 'no_neuron5': 69,
 'no_neuron6': 98,
 'no_neuron7': 82,
 'no_neuron8': 14,
 'no_neuron9': 82}

In [24]:
#rgrid.best_params_['no_neuronX'] feeds neuron output from best params into model
def deepnetwork1():
    model = Sequential()
    model.add(Dense(rgrid.best_params_['no_neuron1'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron2'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron3'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron4'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron5'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron6'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron7'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron8'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron9'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(rgrid.best_params_['no_neuron10'], kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='Adam')
    return model

In [25]:
deep_n=deepnetwork1()
#best params fed directky into model fit
deep_n.fit(X_train,Y_train,epochs=rgrid.best_params_['epochs'],batch_size=rgrid.best_params_['batch_size'],verbose=1)

Epoch 1/32
15/15 [==============================] - 4s 182ms/step - loss: 0.0195
Epoch 2/32
15/15 [==============================] - 3s 166ms/step - loss: 0.0191
Epoch 3/32
15/15 [==============================] - 3s 169ms/step - loss: 0.0191
Epoch 4/32
15/15 [==============================] - 3s 172ms/step - loss: 0.0191
Epoch 5/32
15/15 [==============================] - 4s 241ms/step - loss: 0.0191
Epoch 6/32
15/15 [==============================] - 3s 204ms/step - loss: 0.0191
Epoch 7/32
15/15 [==============================] - 3s 172ms/step - loss: 0.0191
Epoch 8/32
15/15 [==============================] - 3s 178ms/step - loss: 0.0191
Epoch 9/32
15/15 [==============================] - 4s 247ms/step - loss: 0.0191
Epoch 10/32
15/15 [==============================] - 4s 299ms/step - loss: 0.0191
Epoch 11/32
15/15 [==============================] - 4s 258ms/step - loss: 0.0191
Epoch 12/32
15/15 [==============================] - 3s 181ms/step - loss: 0.0191
Epoch 13/32
15/15 [======

In [26]:
Y_predict2=pd.DataFrame(deep_n.predict(X_test),columns=['Y_predict'])

In [27]:
Y_test2=pd.DataFrame(Y_test).reset_index()

In [28]:
Comb1=pd.merge(Y_test2, Y_predict2, left_index=True,right_index=True,how='inner')
Comb1['Year']=Comb1['datadate'].dt.year
Comb1['Month']=Comb1['datadate'].dt.month
rank1=Comb1[['Y_predict','Year', 'Month']].groupby(['Year','Month'],as_index=False).rank(ascending=False)
rank1.rename(columns={'Y_predict':'Y_predict_rank'},inplace=True)
stock_long1=pd.merge(Comb1,rank1,left_index=True, right_index=True)
stock_long2=stock_long1[stock_long1['Y_predict_rank']<=100]
stock_long2['datadate'].value_counts()
stock_long3=stock_long2[['ret','Year','Month']].groupby(['Year','Month']).mean()
stock_long4=pd.merge(stock_long3, rf3, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5=pd.merge(stock_long4, indexret1, left_on=['Year','Month'], right_on=['Year','Month'], how='left')
stock_long5['ret_rf']=stock_long5['ret']-stock_long5['rf']
stock_long5['ret_sp500']=stock_long5['ret']-stock_long5['sp500_ret_m']
stock_long5=sm.add_constant(stock_long5)
sm.OLS(stock_long5[['ret']],stock_long5[['const']]).fit().get_robustcov_results(cov_type='HC0').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 12 Aug 2021   Prob (F-statistic):                nan
Time:                        21:19:44   Log-Likelihood:                 116.31
No. Observations:                  64   AIC:                            -230.6
Df Residuals:                      63   BIC:                            -228.5
Df Model:                           0                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0111      0.005      2.258      0.027       0.001       0.021
==============================================================================
Omnibus:                       67.101   Durbin-Watson:                   1.659
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              718.538
Skew:                          -2.758   Prob(JB):                    9.36e-157
Kurtosis:                      18.461   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC0)
"""

In [ ]:
#1.1% over market

In [29]:
Ret_rfdeepnetwork=stock_long5[['ret_rf']]
SRdeepnetwork=(Ret_rfdeepnetwork.mean()/Ret_rfdeepnetwork.std())*np.sqrt(12)
SRdeepnetwork

ret_rf    0.88788
dtype: float64